In [13]:
import logging
from datetime import datetime
import os

from robin.mesh_hypotheses import (
    mesh_hypotheses_lit_review,
    generate_hypotheses_from_mesh,
    mesh_hypotheses_ranking,
)

from robin.configuration import RobinConfiguration

In [14]:
disease_name="celiac disease" 

def get_timestamp_suffix():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

save_path = f"dark_robin_results/{disease_name.replace(' ', '_')}_{get_timestamp_suffix()}"
os.makedirs(save_path, exist_ok=True)

In [15]:
config = RobinConfiguration(
    disease_name=disease_name,
    llm_name = "claude-sonnet-4", # choose between claude-sonnet-4, claude-opus-4, gemini-2.5-flash, or o4-mini
    num_mesh_hypotheses=5,
    num_queries=2,
    num_assays=2,
    num_candidates=2,
    )


logger = logging.getLogger("robin")
logger.setLevel(logging.INFO)

In [ ]:
from copy import deepcopy
import json

#we dont want to save the API keys in the config file
config_dict = deepcopy(config.model_dump())
for model in config_dict.get("llm_config", {}).get("model_list", []):
    if "api_key" in model.get("litellm_params", {}):
        model["litellm_params"]["api_key"] = None

with open(f"{save_path}/config.json", "w") as f:
    json.dump(config_dict, f, indent=2)

In [4]:
hypotheses = await generate_hypotheses_from_mesh(config)

with open(f"{save_path}/hypotheses.txt", 'w') as f:
    for hypothesis in hypotheses:
        f.write(f"{hypothesis}\n____________________________\n")

2025-06-13 16:42:50,122 - robin.mesh_hypotheses - INFO - 

Step 1: Generating hypotheses from MeSH terms...
2025-06-13 16:43:10,881 - robin.mesh_hypotheses - INFO - Hypotheses Generated from MeSH Terms:
2025-06-13 16:43:10,882 - robin.mesh_hypotheses - INFO - 1. <**Hypothesis**: Viral infections trigger epigenetic modifications in intestinal epithelial cells, altering tight junction protein expression and creating a permeable barrier that facilitates gluten peptide translocation and immune activation in genetically susceptible individuals.

**Rationale**: Recent evidence suggests viral infections can induce lasting epigenetic changes. Rotavirus and other enteric viruses are associated with celiac disease onset, potentially through chromatin remodeling affecting barrier function genes.

**Therapeutic Implications**: Epigenetic modulators like HDAC inhibitors or DNA methyltransferase inhibitors could restore normal tight junction expression, reducing intestinal permeability and preventin

In [5]:
chimp_result = await mesh_hypotheses_lit_review(config, hypotheses)

with open(f'{save_path}/chimp_result.txt', 'w') as f:
    f.write(chimp_result)

2025-06-13 16:43:10,896 - robin.mesh_hypotheses - INFO - 

Step 2: Conducting literature search with FutureHouse platform...
2025-06-13 16:43:11,464 - robin.utils - INFO - Starting literature search for 5 queries using job-futurehouse-chimp.
2025-06-13 16:55:39,333 - robin.utils - INFO - Finished processing 5 tasks.
2025-06-13 16:55:39,338 - robin.utils - INFO - Successfully wrote: query_1_given.txt to: robin_output/celiac_disease_2025-06-13_16-42-49/mesh_hypotheses_literature_reviews/query_1_given.txt
2025-06-13 16:55:39,339 - robin.utils - INFO - Successfully wrote: query_2_given.txt to: robin_output/celiac_disease_2025-06-13_16-42-49/mesh_hypotheses_literature_reviews/query_2_given.txt
2025-06-13 16:55:39,340 - robin.utils - INFO - Successfully wrote: query_3_given.txt to: robin_output/celiac_disease_2025-06-13_16-42-49/mesh_hypotheses_literature_reviews/query_3_given.txt
2025-06-13 16:55:39,341 - robin.utils - INFO - Successfully wrote: query_4_given.txt to: robin_output/celiac_dis

In [6]:
best_hypotheses = await mesh_hypotheses_ranking(config, chimp_result)

with open(f'{save_path}/best_hypotheses.txt', 'w') as f:
    f.write(best_hypotheses)

2025-06-13 16:55:39,347 - robin.mesh_hypotheses - INFO - 

Step 3: Choosing the best hypotheses based on literature review...
